# Binance Trade Data Analysis & Account Ranking

This notebook analyzes Binance trade data over a 90-day period, calculating key financial metrics and ranking accounts based on performance.

In [ ]:

import pandas as pd
import numpy as np

# Function to calculate Maximum Drawdown (MDD)
def calculate_mdd(returns):
    cumulative_returns = (1 + returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    return abs(drawdown.min())  # Keep MDD as a positive value

# Load Dataset
file_path = "TRADES_CopyTr_90D_ROI.csv"
trade_df = pd.read_csv(file_path)

# Convert timestamps to datetime
trade_df["time"] = pd.to_datetime(trade_df["time"], unit="ms")

# Group trades by account
account_stats = trade_df.groupby("Port_IDs").agg(
    total_profit=("realizedProfit", "sum"),  
    total_trades=("Port_IDs", "count"),  
    win_trades=("realizedProfit", lambda x: (x > 0).sum()),  
    total_investment=("quantity", "sum")  
).reset_index()

# Compute additional metrics
account_stats["win_rate"] = (account_stats["win_trades"] / account_stats["total_trades"]) * 100
account_stats["ROI"] = (account_stats["total_profit"] / account_stats["total_investment"]) * 100

# Fill NaN values (if any)
account_stats.fillna(0, inplace=True)


In [ ]:

# Compute daily returns per account
daily_returns = trade_df.groupby(["Port_IDs", trade_df["time"].dt.date])["realizedProfit"].sum().reset_index()
daily_returns.rename(columns={"realizedProfit": "daily_return"}, inplace=True)

# Compute Sharpe Ratio per account
sharpe_ratios = daily_returns.groupby("Port_IDs").agg(
    mean_daily_return=("daily_return", "mean"),
    std_daily_return=("daily_return", "std")
).reset_index()

sharpe_ratios["Sharpe_Ratio"] = sharpe_ratios["mean_daily_return"] / sharpe_ratios["std_daily_return"]
sharpe_ratios.fillna(0, inplace=True)

# Merge Sharpe Ratio with account stats
account_stats = account_stats.merge(sharpe_ratios[["Port_IDs", "Sharpe_Ratio"]], on="Port_IDs", how="left")


In [ ]:

# Normalize daily returns by total investment
daily_returns = daily_returns.merge(account_stats[["Port_IDs", "total_investment"]], on="Port_IDs", how="left")
daily_returns["normalized_return"] = daily_returns["daily_return"] / daily_returns["total_investment"]

# Compute MDD per account
mdd_values = daily_returns.groupby("Port_IDs")["normalized_return"].apply(calculate_mdd).reset_index()
mdd_values.rename(columns={"normalized_return": "MDD"}, inplace=True)

# Merge MDD with account stats
account_stats = account_stats.merge(mdd_values, on="Port_IDs", how="left")


In [ ]:

# Define weights for ranking criteria
weights = {
    "ROI": 0.3, 
    "Sharpe_Ratio": 0.3, 
    "win_rate": 0.2,  
    "total_profit": 0.1,  
    "MDD": -0.1  
}

# Normalize each metric
for metric in weights.keys():
    if weights[metric] > 0:
        account_stats[metric + "_score"] = (account_stats[metric] - account_stats[metric].min()) / (
            account_stats[metric].max() - account_stats[metric].min())
    else:
        account_stats[metric + "_score"] = (account_stats[metric].max() - account_stats[metric]) / (
            account_stats[metric].max() - account_stats[metric].min())

# Compute final ranking score
account_stats["final_score"] = sum(weights[m] * account_stats[m + "_score"] for m in weights.keys())

# Rank accounts by final score
account_stats = account_stats.sort_values(by="final_score", ascending=False)

# Get top 20 accounts
top_20_accounts = account_stats.head(20)

# Save results to CSV
top_20_accounts.to_csv("top_20_accounts.csv", index=False)

# Display top 5 ranked accounts
top_20_accounts.head()
